# 03 — FNO como operador

Exploração da capacidade do FNO em generalizar para novas condições iniciais do conjunto gerado.

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path('..').resolve()
SRC_PATH = PROJECT_ROOT / 'src'
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

In [ ]:
import json
from pathlib import Path
import numpy as np
import torch
from omegaconf import OmegaConf

from riemann_ml.eval.metrics import relative_l2
from riemann_ml.ml.fno.dataset import RiemannH5Dataset, _read_metadata
from riemann_ml.ml.fno.model import FNO1DModel

CONFIG_PATH = Path('src/riemann_ml/configs/fno.yaml')
CHECKPOINT_DIR = Path('data/artifacts/fno/checkpoints')
DATASET_PATH = Path('data/processed/sod_like.h5')

metadata = _read_metadata(DATASET_PATH)
dataset = RiemannH5Dataset(DATASET_PATH)
print(f'Dataset contém {len(dataset)} amostras.')

cfg = OmegaConf.load(CONFIG_PATH)
checkpoint_candidates = sorted(CHECKPOINT_DIR.glob('fno_epoch_*.pt'))
if not checkpoint_candidates:
    raise FileNotFoundError(f'Sem checkpoints em {CHECKPOINT_DIR}')
latest_checkpoint = checkpoint_candidates[-1]
print('Usando checkpoint:', latest_checkpoint)

model = FNO1DModel(OmegaConf.to_container(cfg.model, resolve=True))
device = torch.device(cfg.training.device if torch.cuda.is_available() else 'cpu')
state = torch.load(latest_checkpoint, map_location=device)
state_dict = state.get('model_state', state)
state_dict.pop('_metadata', None)
model.load_state_dict(state_dict)
model.to(device)
model.eval()

sample_idx = int(np.random.randint(0, len(dataset)))
inputs, targets = dataset[sample_idx]
with torch.no_grad():
    preds = model(inputs.unsqueeze(0).to(device)).cpu().numpy()[0]

rho_true, u_true, p_true = targets.numpy()
rho_pred, u_pred, p_pred = preds
x = metadata.x

r_l2 = relative_l2(rho_pred, rho_true)
u_l2 = relative_l2(u_pred, u_true)
p_l2 = relative_l2(p_pred, p_true)
print(f'Relative L2 (rho) = {r_l2:.4f}, (u) = {u_l2:.4f}, (p) = {p_l2:.4f}')

import matplotlib.pyplot as plt
fig, axes = plt.subplots(3, 1, figsize=(10, 9), sharex=True)
axes[0].plot(x, rho_true, label='True')
axes[0].plot(x, rho_pred, '--', label='FNO')
axes[0].set_ylabel('Density')
axes[0].grid(True)
axes[0].legend()

axes[1].plot(x, u_true, label='True')
axes[1].plot(x, u_pred, '--', label='FNO')
axes[1].set_ylabel('Velocity')
axes[1].grid(True)

axes[2].plot(x, p_true, label='True')
axes[2].plot(x, p_pred, '--', label='FNO')
axes[2].set_ylabel('Pressure')
axes[2].set_xlabel('x')
axes[2].grid(True)

fig.suptitle(f'FNO vs referência (sample {sample_idx})')
fig.tight_layout()
plt.show()


## Métricas em lote

Consulta dos resultados agregados gerados pelo script de avaliação, úteis para comentar generalização.


In [ ]:
import json
from pathlib import Path

metrics_path = Path('data/artifacts/eval/dataset/dataset_metrics.json')
if metrics_path.exists():
    metrics = json.loads(metrics_path.read_text(encoding='utf-8'))
    print(f'Total de amostras avaliadas: {len(metrics)}')
    if metrics:
        first = metrics[0]
        print('Exemplo de métricas por modelo para a primeira amostra:')
        for model_name, values in first['metrics'].items():
            print(f'  {model_name}:')
            for key, value in values.items():
                print(f'    {key}: {value}')
else:
    print('Arquivo de métricas não encontrado:', metrics_path)


> **Comentário:** use os valores acima para discutir como o FNO performa fora da condição Sod e quais faixas de erro são aceitáveis.
